# 004_join01

In [1]:
library(dplyr)
source('preprocess/load_data/data_loader.R')
load_hotel_reserve()

# 下記から本書掲載
# reserve_tableとhotel_tbをhotel_idが等しいデータ同士で内部結合
inner_join(reserve_tb, hotel_tb, by='hotel_id') %>%
  
  # people_numが1かつis_businessがTrueのデータのみ抽出
  filter(people_num == 1, is_business)



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,34500,C,C-1,38.23729,140.6961,TRUE
r11,h_183,c_2,2016-11-19 12:49:10,2016-12-08,11:00:00,2016-12-11,1,29700,9900,G,G-4,33.59525,130.6336,TRUE
r13,h_223,c_2,2017-10-19 03:03:30,2017-10-21,09:30:00,2017-10-23,1,137000,68500,C,C-2,38.32910,140.6982,TRUE
r18,h_132,c_3,2016-10-22 02:18:48,2016-11-12,12:00:00,2016-11-13,1,20400,20400,C,C-1,38.23184,140.7973,TRUE
r25,h_277,c_4,2016-03-28 07:17:34,2016-04-07,10:30:00,2016-04-10,1,39300,13100,C,C-1,38.23399,140.7956,TRUE
r26,h_132,c_4,2016-05-11 17:48:07,2016-06-05,11:30:00,2016-06-06,1,20400,20400,C,C-1,38.23184,140.7973,TRUE
r32,h_287,c_4,2017-11-02 19:00:21,2017-11-05,10:00:00,2017-11-07,1,29000,14500,G,G-4,33.59628,130.6386,TRUE
r34,h_273,c_5,2016-11-25 15:44:32,2016-12-19,12:00:00,2016-12-22,1,134700,44900,A,A-3,35.80955,139.9407,TRUE
r35,h_90,c_5,2017-03-30 13:38:03,2017-04-05,11:30:00,2017-04-07,1,16000,8000,A,A-3,35.91594,139.9318,TRUE
r42,h_63,c_7,2016-11-11 12:42:32,2016-11-26,09:00:00,2016-11-29,1,44700,14900,B,B-1,35.44331,139.5954,TRUE


In [2]:
library(dplyr)
source('preprocess/load_data/data_loader.R')
load_hotel_reserve()

# 下記から本書掲載
inner_join(reserve_tb %>% filter(people_num == 1),
           hotel_tb %>% filter(is_business),
           by='hotel_id')


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,34500,C,C-1,38.23729,140.6961,TRUE
r11,h_183,c_2,2016-11-19 12:49:10,2016-12-08,11:00:00,2016-12-11,1,29700,9900,G,G-4,33.59525,130.6336,TRUE
r13,h_223,c_2,2017-10-19 03:03:30,2017-10-21,09:30:00,2017-10-23,1,137000,68500,C,C-2,38.32910,140.6982,TRUE
r18,h_132,c_3,2016-10-22 02:18:48,2016-11-12,12:00:00,2016-11-13,1,20400,20400,C,C-1,38.23184,140.7973,TRUE
r25,h_277,c_4,2016-03-28 07:17:34,2016-04-07,10:30:00,2016-04-10,1,39300,13100,C,C-1,38.23399,140.7956,TRUE
r26,h_132,c_4,2016-05-11 17:48:07,2016-06-05,11:30:00,2016-06-06,1,20400,20400,C,C-1,38.23184,140.7973,TRUE
r32,h_287,c_4,2017-11-02 19:00:21,2017-11-05,10:00:00,2017-11-07,1,29000,14500,G,G-4,33.59628,130.6386,TRUE
r34,h_273,c_5,2016-11-25 15:44:32,2016-12-19,12:00:00,2016-12-22,1,134700,44900,A,A-3,35.80955,139.9407,TRUE
r35,h_90,c_5,2017-03-30 13:38:03,2017-04-05,11:30:00,2017-04-07,1,16000,8000,A,A-3,35.91594,139.9318,TRUE
r42,h_63,c_7,2016-11-11 12:42:32,2016-11-26,09:00:00,2016-11-29,1,44700,14900,B,B-1,35.44331,139.5954,TRUE


# 004_join02

In [3]:
library(dplyr)
source('preprocess/load_data/data_loader.R')
load_hotel_reserve()

# 下記から本書掲載
# small_area_nameごとにホテル数をカウント、結合キーを判定するためのテーブル
small_area_mst <-
  hotel_tb %>%
    group_by(big_area_name, small_area_name) %>%

    # -1は、自ホテルを引いている
    summarise(hotel_cnt=n() - 1) %>%

    # 集約処理完了後に、グループ化を解除
    ungroup() %>%

    # 20件以上であればjoin_area_idをsmall_area_nameとして設定
    # 20件未満であればjoin_area_idをbig_area_nameとして設定
    mutate(join_area_id=
             if_else(hotel_cnt >= 20, small_area_name, big_area_name)) %>%
    select(small_area_name, join_area_id)

# レコメンド元になるホテルにsmall_area_mstを結合することで、join_area_idを設定
base_hotel_mst <-
  inner_join(hotel_tb, small_area_mst, by='small_area_name') %>%
    select(hotel_id, join_area_id)

# 必要に応じて、メモリを解放(必須ではないがメモリ量に余裕のないときに利用)
rm(small_area_mst)

# recommend_hotel_mstはレコメンド候補のためのテーブル
recommend_hotel_mst <-
  bind_rows(
    # join_area_idをbig_area_nameとしたレコメンド候補マスタ
    hotel_tb %>%
      rename(rec_hotel_id=hotel_id, join_area_id=big_area_name) %>%
      select(join_area_id, rec_hotel_id),

    # join_area_idをsmall_area_nameとしたレコメンド候補マスタ
    hotel_tb %>%
      rename(rec_hotel_id=hotel_id, join_area_id=small_area_name) %>%
      select(join_area_id, rec_hotel_id)
  )

# base_hotel_mstとrecommend_hotel_mstを結合し、レコメンド候補の情報を付与
inner_join(base_hotel_mst, recommend_hotel_mst, by='join_area_id') %>%

  # レコメンド候補から自分を除く
  filter(hotel_id != rec_hotel_id) %>%
  select(hotel_id, rec_hotel_id)


hotel_id,rec_hotel_id
h_1,h_14
h_1,h_22
h_1,h_27
h_1,h_40
h_1,h_45
h_1,h_77
h_1,h_79
h_1,h_85
h_1,h_91
h_1,h_103


# 004_join03_a

In [4]:
library(dplyr)
source('preprocess/load_data/data_loader.R')
load_hotel_reserve()

# 下記から本書掲載
reserve_tb %>%

  # group_byによって、customer_idごとにデータをグループ化
  group_by(customer_id) %>%

  # LAG関数を利用し、2件前のtotal_priceをbefore_priceとして取得
  # LAG関数によって参照する際のグループ内のデータをreserve_datetimeの古い順に指定
  mutate(before_price=lag(total_price, n=2,
                          order_by=reserve_datetime, default=NA))


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,before_price
r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,NA
r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,NA
r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,97200
r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,20600
r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,33600
r6,h_241,c_1,2017-11-27 18:47:05,2017-12-04,12:00:00,2017-12-06,3,36000,194400
r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,68100
r8,h_241,c_1,2018-05-26 08:42:51,2018-06-08,10:00:00,2018-06-09,1,6000,36000
r9,h_217,c_2,2016-03-05 13:31:06,2016-03-25,09:30:00,2016-03-27,3,68400,NA
r10,h_240,c_2,2016-06-25 09:12:22,2016-07-14,11:00:00,2016-07-17,4,320400,NA


# 004_join03_b

In [5]:
library(dplyr)
source('preprocess/load_data/data_loader.R')
load_hotel_reserve()

# 下記から本書掲載
# roll_sum関数のためのライブラリ
library(RcppRoll)

reserve_tb %>%

  # データ行をcustomer_idごとにグループ化
  group_by(customer_id) %>%

  # customer_idごとにreserve_datetimeでデータを並び替え
  arrange(reserve_datetime) %>%

  # RcppRollのroll_sumによって、移動合計値を計算
  mutate(price_sum=roll_sum(total_price, n=3, align='right', fill=NA))


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_sum
r2757,h_169,c_696,2016-01-01 08:56:43,2016-01-15,11:30:00,2016-01-18,4,100800,NA
r1974,h_156,c_493,2016-01-01 09:40:10,2016-01-26,09:00:00,2016-01-28,1,36800,NA
r2646,h_142,c_665,2016-01-01 11:07:09,2016-01-02,09:00:00,2016-01-05,4,111600,NA
r2444,h_129,c_611,2016-01-01 11:25:41,2016-01-26,12:30:00,2016-01-27,2,16000,NA
r3286,h_241,c_826,2016-01-01 12:20:12,2016-01-02,09:30:00,2016-01-04,2,24000,NA
r1920,h_299,c_475,2016-01-01 19:35:25,2016-01-13,12:30:00,2016-01-16,3,217800,NA
r3205,h_65,c_806,2016-01-02 01:07:26,2016-01-16,10:30:00,2016-01-17,3,25200,NA
r927,h_109,c_227,2016-01-02 02:54:18,2016-01-10,10:30:00,2016-01-12,4,84000,NA
r479,h_179,c_110,2016-01-02 05:24:11,2016-01-21,10:30:00,2016-01-24,3,50400,NA
r1819,h_236,c_450,2016-01-02 18:03:54,2016-01-31,10:30:00,2016-02-03,2,93000,NA


# 004_join03_c

In [6]:
library(dplyr)
source('preprocess/load_data/data_loader.R')
load_hotel_reserve()

# 下記から本書掲載
# row_number関数でreserve_datetimeを利用するために、POSIXct型に変換
# (「第10章 日時型」で詳しく解説)
reserve_tb$reserve_datetime <-
  as.POSIXct(reserve_tb$reserve_datetime, format='%Y-%m-%d %H:%M:%S')

reserve_tb %>%
  group_by(customer_id) %>%
  arrange(reserve_datetime) %>%

  # １〜3件前のtotal_priceの合計をlag関数によって計算
  # if_else関数とrank関数を組み合わせて、何件合計したかを判定
  # order_by=reserve_datetimeの指定は、事前に並び替えられているので必須ではない
  # 合計した件数が0だった場合、0で割っているためprice_avgがNANとなる
  mutate(price_avg=
           (  lag(total_price, 1, order_by=reserve_datetime, default=0)
            + lag(total_price, 2, order_by=reserve_datetime, default=0)
            + lag(total_price, 3, order_by=reserve_datetime, default=0))
           / if_else(row_number(reserve_datetime) > 3,
                     3, row_number(reserve_datetime) - 1))


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_avg
r2757,h_169,c_696,2016-01-01 08:56:43,2016-01-15,11:30:00,2016-01-18,4,100800,NaN
r1974,h_156,c_493,2016-01-01 09:40:10,2016-01-26,09:00:00,2016-01-28,1,36800,NaN
r2646,h_142,c_665,2016-01-01 11:07:09,2016-01-02,09:00:00,2016-01-05,4,111600,NaN
r2444,h_129,c_611,2016-01-01 11:25:41,2016-01-26,12:30:00,2016-01-27,2,16000,NaN
r3286,h_241,c_826,2016-01-01 12:20:12,2016-01-02,09:30:00,2016-01-04,2,24000,NaN
r1920,h_299,c_475,2016-01-01 19:35:25,2016-01-13,12:30:00,2016-01-16,3,217800,NaN
r3205,h_65,c_806,2016-01-02 01:07:26,2016-01-16,10:30:00,2016-01-17,3,25200,NaN
r927,h_109,c_227,2016-01-02 02:54:18,2016-01-10,10:30:00,2016-01-12,4,84000,NaN
r479,h_179,c_110,2016-01-02 05:24:11,2016-01-21,10:30:00,2016-01-24,3,50400,NaN
r1819,h_236,c_450,2016-01-02 18:03:54,2016-01-31,10:30:00,2016-02-03,2,93000,NaN


# 004_join03_d

In [7]:
library(dplyr)
source('preprocess/load_data/data_loader.R')
load_hotel_reserve()

# 下記から本書掲載
library(tidyr)

# row_number関数でreserve_datetimeを利用するために、POSIXct型に変換
# (「第10章 日時型」で詳しく解説)
reserve_tb$reserve_datetime <-
  as.POSIXct(reserve_tb$reserve_datetime, format='%Y-%m-%d %H:%M:%S')

# 過去90日間の合計予約金額を計算したテーブル
sum_table <-

  # reserve_datetimeの日付を確認せずに、同じcustomer_idのデータ行同士をすべて結合
  inner_join(
    reserve_tb %>%
      select(reserve_id, customer_id, reserve_datetime),
    reserve_tb %>%
      select(customer_id, reserve_datetime, total_price) %>%
      rename(reserve_datetime_before=reserve_datetime),
    by='customer_id') %>%

  # checkinの日付を比較して、90日以内のデータが結合されているデータ行のみ抽出
  # 60*60*24*90は、60秒*60分*24時間*90日を意味し、90日間分の秒数を計算
  # (日付のデータ型については、「第10章 日時型」で詳しく解説)
  filter(reserve_datetime > reserve_datetime_before &
           reserve_datetime - 60 * 60 * 24 * 90 <= reserve_datetime_before) %>%
  select(reserve_id, total_price) %>%

  # reserve_idごとにtotal_priceの合計値を計算
  group_by(reserve_id) %>%
  summarise(total_price_90d=sum(total_price)) %>%
  select(reserve_id, total_price_90d)

# 計算した合計値を結合し、元のテーブルに情報を付与
# 合計値が存在しないレコードの合計値の値を、replace_naを利用して0に変更
left_join(reserve_tb, sum_table, by='reserve_id') %>%
  replace_na(list(total_price_90d=0))


reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,total_price_90d
r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,0
r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,0
r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,20600
r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,0
r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,0
r6,h_241,c_1,2017-11-27 18:47:05,2017-12-04,12:00:00,2017-12-06,3,36000,68100
r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,36000
r8,h_241,c_1,2018-05-26 08:42:51,2018-06-08,10:00:00,2018-06-09,1,6000,0
r9,h_217,c_2,2016-03-05 13:31:06,2016-03-25,09:30:00,2016-03-27,3,68400,0
r10,h_240,c_2,2016-06-25 09:12:22,2016-07-14,11:00:00,2016-07-17,4,320400,0


# 004_join04

In [8]:
library(dplyr)
source('preprocess/load_data/data_loader.R')
load_hotel_reserve()

# 下記から本書掲載
library(tidyverse)

# 計算対象の年月のデータフレームを作成
month_mst <- data.frame(year_month=
  # 2017-01-01、2017-02-01, 2017-03-01を生成し、format関数で形式を年月に変換
  # (日付のデータ型については、「第10章 日時型」で詳しく解説)
  format(seq(as.Date('2017-01-01'), as.Date('2017-03-01'), by='months'),
         format='%Y%m')
)

# 顧客IDと計算対象のすべての年月が結合したテーブル
customer_mst <-

  # すべての顧客IDと年月マスタを全結合
  merge(customer_tb %>% select(customer_id), month_mst) %>%

  # mergeで指定した結合キーのデータ型がカテゴリ型になっているので、文字型に戻す
  # (カテゴリ型については、「第9章 カテゴリ型」で詳しく解説)
  mutate(customer_id=as.character(customer_id),
         year_month=as.character(year_month))

# 合計利用金額を月ごとに計算
left_join(
  customer_mst,

  # 予約テーブルに年月の結合キーを準備
  reserve_tb %>%
    mutate(checkin_month = format(as.Date(checkin_date), format='%Y%m')),

  # 同じcustomer_idと年月を結合
  by=c('customer_id'='customer_id', 'year_month'='checkin_month')
) %>%

  # customer_idと年月で集約
  group_by(customer_id, year_month) %>%

  # 合計金額を算出
  summarise(price_sum=sum(total_price)) %>%

  # 予約レコードがなかった場合の合計金額を値なしから0に変換
  replace_na(list(price_sum=0))


── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0     ✔ purrr   0.3.2
✔ tibble  2.1.1     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


customer_id,year_month,price_sum
c_1,201701,0
c_1,201702,0
c_1,201703,194400
c_10,201701,0
c_10,201702,0
c_10,201703,0
c_100,201701,0
c_100,201702,18000
c_100,201703,0
c_1000,201701,0
